# Задание 6. Классификация новостей

### Данные
Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тестовое множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

## Задание 6.1 

Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):

- pymorphy2
- русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
- [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
## Задание 6.2

Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

## Задание 6.3

Реализовать алгоритм классификации документа по категориям, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
- SVM
- наивный байесовский классификатор
- логистическая регрессия
    

## Задание 6.4* 

Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
%pip install -q gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import pandas as pd
import numpy as np
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn import naive_bayes

In [12]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Viktor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
train_data = pd.read_csv(r'../data/news_train.txt', sep="\t", header=None)
train_data.rename(columns = {0: 'mark', 1: 'header', 2: 'text'}, inplace=True)
test_data = pd.read_csv(r'../data/news_test.txt', sep="\t", header=None)
test_data.rename(columns = {0: 'mark', 1: 'header', 2: 'text'}, inplace=True)

In [20]:
train_data.head()

,mark,header,text
0,sport,Овечкин пожертвовал детской хоккейной школе ав...,Нападающий «Вашингтон Кэпиталз» Александр Овеч...
1,culture,Рекордно дорогую статую майя признали подделкой,"Власти Мексики объявили подделкой статую майя,..."
2,science,Samsung представила флагман в защищенном корпусе,Южнокорейская Samsung анонсировала защищенную ...
3,sport,С футболиста «Спартака» сняли четырехматчевую ...,Контрольно-дисциплинарный комитет (КДК) РФС сн...
4,media,Hopes & Fears объединится с The Village,Интернет-издание Hopes & Fears объявило о свое...


In [13]:
stemmer = SnowballStemmer("russian")

In [21]:
def clean_text(text, num):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = newString.replace('[', '')
    newString = newString.replace(']', '')
    newString = re.sub('»','', newString)
    newString = re.sub('«','', newString)
    newString = re.sub('"','', newString)
    newString = re.sub(r'[,"\'-?:!;—%]', '', newString)
    if(num==0):
        tokens = [w for w in newString.split()]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:
            long_words.append(stemmer.stem(i))
    return (" ".join(long_words)).strip()

In [22]:
cleaned_header_train = []
cleaned_text_train = []

for t in train_data['header']:
    cleaned_header_train.append(clean_text(t, 0))

for t in train_data['text']:
    cleaned_text_train.append(clean_text(t, 0))

print(cleaned_text_train[:5])

['напада вашингтон кэпиталз александр овечкин переда детск хоккейн школ автомобил получен им посл окончан матч всех звезд национальн хоккейн лиг об эт сообща на официальн сайт лигиавтомобил honda accord был подар хоккеист по решен спонсор мероприят игрок нхл пожертвова машин спортивн школ nova cool cats special hockey inc котор располож штат вирджинияовечкин обща летн девочк ан шоб синдром даун котор занима эт школ явля поклонниц спортсм сентябр форвард пообеда вмест юн хоккеистк японск ресторанематч всех звезд нхл коламбус заверш побед команд джоната тэйвз над команд ник фолин со счет овечкин выступа за проигра коллект россиянин отмет трем результативн передач', 'власт мексик объяв подделк стат май прода на эт недел на аукцион париж за рекордн сумм миллион евр сообща agence франcепрессеминистерств иностра дел национальн институт антрополог истор выступ совместн заявлен утвержд что стату не относ ни одн из доколумбов культур центральн америк утвержда что произведен относ позднеклассиче

In [23]:
marks = train_data.mark.values
header = cleaned_header_train
text = cleaned_text_train
columns = ['mark', 'header', 'text']
arr = []
for i in range(len(marks)):
    arr.append([marks[i], header[i], text[i]])
new_df = pd.DataFrame(arr, columns=columns)
print(new_df)

            mark                                             header  \
0          sport    овечкин пожертвова детск хоккейн школ автомобил   
1        culture              рекордн дорог стат май призна подделк   
2        science             samsung представ флагма защищен корпус   
3          sport  футболист спартак снял четырехматчев дисквалиф...   
4          media                    hopes fears объедин the village   
...          ...                                                ...   
14995       life            составл рейтинг лучш европейск пляж год   
14996      media                     сноб объясн причин смен формат   
14997  economics   минфин предлож штрафова за биткоин на тысяч рубл   
14998       life         мэл гибсон заплат бывш подруг тысяч доллар   
14999      media         ещ на двух лин московск метр заработа wifi   

                                                    text  
0      напада вашингтон кэпиталз александр овечкин пе...  
1      власт мексик объяв под

In [24]:
model = Word2Vec()
titles= new_df.header.apply(lambda x : x.split(" ")).values
sentences = new_df.text.apply(lambda x : x.split(" ")).values
model.build_vocab(titles + sentences)

In [25]:
model.train(titles + sentences, total_examples=len(sentences), epochs=20)

(43781119, 50385420)

In [34]:
model.wv.key_to_index

{'на': 0,
 'что': 1,
 'по': 2,
 'год': 3,
 'эт': 4,
 'не': 5,
 'котор': 6,
 'был': 7,
 'из': 8,
 'он': 9,
 'за': 10,
 'для': 11,
 'как': 12,
 'росс': 13,
 'сообща': 14,
 'об': 15,
 'сво': 16,
 'такж': 17,
 'от': 18,
 'ег': 19,
 'компан': 20,
 'до': 21,
 'так': 22,
 'российск': 23,
 'нов': 24,
 'будет': 25,
 'перв': 26,
 'процент': 27,
 'врем': 28,
 'посл': 29,
 'миллион': 30,
 'доллар': 31,
 'стал': 32,
 'заяв': 33,
 'со': 34,
 'сам': 35,
 'слов': 36,
 'при': 37,
 'тем': 38,
 'одн': 39,
 'е': 40,
 'тысяч': 41,
 'друг': 42,
 'дан': 43,
 'тог': 44,
 'сайт': 45,
 'миллиард': 46,
 'рубл': 47,
 'получ': 48,
 'стран': 49,
 'том': 50,
 'во': 51,
 'работ': 52,
 'явля': 53,
 'игр': 54,
 'но': 55,
 'однак': 56,
 'бол': 57,
 'сша': 58,
 'их': 59,
 'лет': 60,
 'the': 61,
 'фильм': 62,
 'проект': 63,
 'сообщ': 64,
 'уж': 65,
 'больш': 66,
 'мир': 67,
 'все': 68,
 'мест': 69,
 'нача': 70,
 'ил': 71,
 'отмет': 72,
 'может': 73,
 'нескольк': 74,
 'то': 75,
 'состав': 76,
 'под': 77,
 'суд': 78,
 'амер

In [39]:
model.wv.most_similar('женщин')

[('мужчин', 0.8007836937904358),
 ('девушк', 0.6749954223632812),
 ('девочк', 0.6705344915390015),
 ('девушек', 0.648724377155304),
 ('люд', 0.6315656304359436),
 ('дет', 0.6170727014541626),
 ('подростк', 0.6149153709411621),
 ('курен', 0.6059340238571167),
 ('родител', 0.599683940410614),
 ('ребенк', 0.580666184425354)]

In [51]:
model.wv.most_similar('мужчин')

[('женщин', 0.8007838129997253),
 ('девушк', 0.7070478796958923),
 ('полицейск', 0.6713160872459412),
 ('девочк', 0.6541179418563843),
 ('подростк', 0.6533882021903992),
 ('водител', 0.6074378490447998),
 ('девушек', 0.6031743288040161),
 ('мальчик', 0.5883130431175232),
 ('преступник', 0.5880602598190308),
 ('родител', 0.5644052624702454)]

In [55]:
lr_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('lr_clf', LogisticRegression(random_state=42))])
lr_ppl_clf.fit(train_data.text, train_data.mark)

c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('lr_clf', LogisticRegression(random_state=42))])

In [56]:
predicted_lr = lr_ppl_clf.predict(test_data.text)
print(metrics.classification_report(predicted_lr, test_data.mark))

              precision    recall  f1-score   support

    business       0.23      0.78      0.36        27
     culture       0.92      0.92      0.92       427
   economics       0.91      0.82      0.87       473
      forces       0.88      0.83      0.85       261
        life       0.90      0.81      0.86       461
       media       0.88      0.84      0.86       422
     science       0.86      0.89      0.88       451
       sport       0.97      0.97      0.97       421
       style       0.71      1.00      0.83        37
      travel       0.37      1.00      0.54        20

    accuracy                           0.87      3000
   macro avg       0.76      0.89      0.79      3000
weighted avg       0.89      0.87      0.88      3000



In [60]:
lr_params = {'lr_clf__C': [10, 1.0, 0.1, 0.01, 0.001],
            }
lr_grid = GridSearchCV(estimator=lr_ppl_clf, param_grid=lr_params)
lr_grid.fit(train_data.text, train_data.mark)

c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver option

GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('lr_clf',
                                        LogisticRegression(random_state=42))]),
             param_grid={'lr_clf__C': [10, 1.0, 0.1, 0.01, 0.001]})

In [61]:
print(lr_grid.best_params_, lr_grid.best_score_)

{'lr_clf__C': 10} 0.8710666666666667


In [62]:
predicted_lr_grid = lr_grid.predict(test_data.text)
print(metrics.classification_report(predicted_lr_grid, test_data.mark))

              precision    recall  f1-score   support

    business       0.48      0.78      0.59        55
     culture       0.93      0.93      0.93       428
   economics       0.92      0.87      0.89       448
      forces       0.89      0.82      0.86       266
        life       0.92      0.85      0.88       446
       media       0.88      0.87      0.87       407
     science       0.86      0.90      0.88       449
       sport       0.97      0.98      0.98       419
       style       0.81      0.95      0.88        44
      travel       0.65      0.92      0.76        38

    accuracy                           0.89      3000
   macro avg       0.83      0.89      0.85      3000
weighted avg       0.90      0.89      0.89      3000



In [63]:
mnb_ppl_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('mnb_clf', naive_bayes.MultinomialNB())])
mnb_ppl_clf.fit(train_data.text, train_data.mark)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('mnb_clf', MultinomialNB())])

In [64]:
predicted_mnb = mnb_ppl_clf.predict(test_data.text)
print(metrics.classification_report(predicted_mnb, test_data.mark))

              precision    recall  f1-score   support

    business       0.00      0.00      0.00         0
     culture       0.92      0.87      0.89       450
   economics       0.97      0.66      0.78       629
      forces       0.51      0.93      0.65       134
        life       0.77      0.87      0.82       368
       media       0.87      0.73      0.79       480
     science       0.87      0.81      0.84       503
       sport       0.99      0.96      0.97       436
       style       0.00      0.00      0.00         0
      travel       0.00      0.00      0.00         0

    accuracy                           0.81      3000
   macro avg       0.59      0.58      0.57      3000
weighted avg       0.89      0.81      0.84      3000



c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this beh

In [65]:
parameters = {'mnb_clf__alpha': [0, 1]}
mnb_grid = GridSearchCV(mnb_ppl_clf, parameters)
mnb_grid.fit(train_data.text, train_data.mark)

c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\naive_bayes.py:627: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\naive_bayes.py:627: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
c:\Users\Viktor\source\clone\classification\classification-2022-3\.venv\lib\site-packages\sklearn\naive_bayes.py:633: UserWarning: alpha too small will result in nume

GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('mnb_clf', MultinomialNB())]),
             param_grid={'mnb_clf__alpha': [0, 1]})

In [66]:
print(mnb_grid.best_params_, mnb_grid.best_score_)

{'mnb_clf__alpha': 0} 0.8132666666666666


In [67]:
predicted_mnb_grid = mnb_grid.predict(test_data.text)
print(metrics.classification_report(predicted_mnb_grid, test_data.mark))

              precision    recall  f1-score   support

    business       0.21      0.61      0.31        31
     culture       0.89      0.85      0.87       445
   economics       0.89      0.82      0.85       464
      forces       0.76      0.81      0.78       232
        life       0.83      0.79      0.81       438
       media       0.81      0.72      0.76       454
     science       0.83      0.80      0.82       480
       sport       0.94      0.99      0.96       402
       style       0.56      0.91      0.69        32
      travel       0.41      1.00      0.58        22

    accuracy                           0.82      3000
   macro avg       0.71      0.83      0.74      3000
weighted avg       0.84      0.82      0.83      3000

